# TACH3

this notebook contains the following:

    NetworkX graph generation

    graph metrics
    
    centrality measures

In [1]:
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as plt
import re
from random import sample
import graphdb
import pandas as pd
import numpy as np

from SPARQLWrapper import SPARQLWrapper, JSON

from pyvis.network import Network
import random

import requests

import urllib.request

import urllib3

import urllib.parse

import json 

from rdflib import Graph

import operator

import networkx as nx

In [2]:
sparql = SPARQLWrapper(
    "http://LAPTOP-CS19OKIS:7200/repositories/tach4"
)
sparql.setReturnFormat(JSON)

def answer_df(query):

    sparql.setQuery(query)

    try:
        ret = sparql.queryAndConvert()     
    except Exception as e:
        print(e)

    dataframe=pd.DataFrame(ret["results"]["bindings"])
    cleandata={}

    for l in dataframe.columns:
        cleandata[l]=[]
    
    for r in ret["results"]["bindings"]:
        for l in dataframe.columns:
            cleandata[l].append(r[l]["value"])

    dataframe=pd.DataFrame(cleandata)
    return dataframe

In [3]:
queryString="""
SELECT DISTINCT ?o ?p ?s WHERE{
    ?o ?p ?s.
}

"""

df=answer_df(queryString)
df

,o,p,s
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
1,http://www.w3.org/2000/01/rdf-schema#subProper...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
2,http://www.w3.org/2000/01/rdf-schema#subProper...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#TransitiveProperty
3,http://www.w3.org/2000/01/rdf-schema#subClassOf,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
4,http://www.w3.org/2000/01/rdf-schema#subClassOf,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#TransitiveProperty
...,...,...,...
815973,http://krono.act.uji.es/schemas/slodbi#AjdeCas...,http://krono.act.uji.es/schemas/slodbi#web,http://www.castellfort.es
815974,http://krono.act.uji.es/schemas/slodbi#AytoBen...,http://krono.act.uji.es/schemas/slodbi#web,http://www.benicassim.org
815975,http://krono.act.uji.es/schemas/slodbi#NulesInfo,http://krono.act.uji.es/schemas/slodbi#web,http://www.nules.es
815976,http://krono.act.uji.es/schemas/slodbi#AjtVila...,http://krono.act.uji.es/schemas/slodbi#web,https://vilafames.org


In [4]:
def cleanDataFrame(df):

    
    dataset=df
    
    if 'p' in dataset.columns:
        listeu=dataset["p"]
        dataset=dataset[-dataset["o"].isin(listeu)]
    
        dataset=dataset[-dataset["o"].str.match(".*www.w3.org.*")]
    
    pat = r'.*\/(.)'
    dataset["s"]= dataset["s"].str.replace(pat,r'\1',regex=True)
    dataset["o"]= dataset["o"].str.replace(pat,r'\1',regex=True)
    

    
    dataset["o"]=dataset["o"].str.replace(".*#(.)",r'\1',regex=True)
    dataset["s"]=dataset["s"].str.replace(".*#(.)",r'\1',regex=True)
    
    """
    lab=dataset[dataset["p"].isin(["http://www.w3.org/2000/01/rdf-schema#label","http://www.w3.org/2006/vcard/ns#fn"])]
    
    dataset=pd.concat([lab, dataset]).drop_duplicates(keep=False)
    labels=lab["s"].tolist()
    urls=lab["o"].tolist()
    #dataset=dataset.replace(urls,labels) non efficace pour un gros dataset
    
    replaceDic = dict(zip(urls, labels))
    dataset["o"] = dataset["o"].map(lambda x: replaceDic.get(x,x))
    dataset["s"] = dataset["s"].map(lambda x: replaceDic.get(x,x))
    """

    return dataset


In [5]:
cleandf=cleanDataFrame(df)
cleandf

,o,p,s
34,Ain,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Municipio
38,hotel-7a2575c0543108e27c8ebf3cdf705d98eb620f18,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,hotel
42,Comunidad_Valenciana,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,ComunidadAutonoma
44,Plana_Baja,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Comarca
46,Castellon,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Provincia
...,...,...,...
815973,AjdeCastellfort,http://krono.act.uji.es/schemas/slodbi#web,www.castellfort.es
815974,AytoBenicassim,http://krono.act.uji.es/schemas/slodbi#web,www.benicassim.org
815975,NulesInfo,http://krono.act.uji.es/schemas/slodbi#web,www.nules.es
815976,AjtVilafames,http://krono.act.uji.es/schemas/slodbi#web,vilafames.org


In [6]:
typesQuery="""
SELECT DISTINCT ?o ?p ?s WHERE{
    ?o <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?s.
}

"""

types=answer_df(typesQuery)
types=cleanDataFrame(types)

types

,o,s
0,type,Property
1,subPropertyOf,Property
2,subPropertyOf,TransitiveProperty
3,subClassOf,Property
4,subClassOf,TransitiveProperty
...,...,...
82398,genid-74817f740289417eb03e454f490779cf-E7C843E...,Voice
82399,genid-74817f740289417eb03e454f490779cf-AA792CB...,Point
82400,genid-74817f740289417eb03e454f490779cf-F87FC4E...,Voice
82401,genid-74817f740289417eb03e454f490779cf-7B7D52B...,Point


In [7]:
typesdict=types.set_index("o")["s"].to_dict()



len(typesdict)

82392

In [8]:
typesdict2=cleandf[cleandf["p"]=="http://www.w3.org/1999/02/22-rdf-syntax-ns#type"][["o","s"]].set_index('o')['s'].to_dict()
len(typesdict2)

82321

In [9]:
set1 = set(typesdict.items())
set2 = set(typesdict2.items())
set1 ^ set2

{('XMLLiteral', 'Datatype'),
 ('_1', 'ContainerMembershipProperty'),
 ('asWKT', 'Property'),
 ('autonomia', 'Property'),
 ('categoria-alojamiento', 'Property'),
 ('comarca', 'Property'),
 ('comment', 'Property'),
 ('differentFrom', 'SymmetricProperty'),
 ('domain', 'Property'),
 ('equivalentClass', 'TransitiveProperty'),
 ('equivalentProperty', 'TransitiveProperty'),
 ('first', 'Property'),
 ('fn', 'Property'),
 ('followers_count', 'Property'),
 ('follows', 'Property'),
 ('friends_count', 'Property'),
 ('generated', 'Property'),
 ('habitaciones', 'Property'),
 ('hadMember', 'Property'),
 ('hasAddress', 'Property'),
 ('hasEmail', 'Property'),
 ('hasProv', 'Property'),
 ('hasTelephone', 'Property'),
 ('hasValue', 'Property'),
 ('inverseOf', 'SymmetricProperty'),
 ('label', 'Property'),
 ('locality', 'Property'),
 ('name', 'Property'),
 ('nil', 'List'),
 ('nonNegativeInteger', 'Datatype'),
 ('num-agencias-de-viaje-central', 'Property'),
 ('num-agencias-de-viaje-sucursal', 'Property'),
 ('

these nodes are not interesting, using the second method won't cause loss of information as these are nodes we already got rid of

In [10]:
nodeTypes=cleandf[cleandf["p"]=="http://www.w3.org/1999/02/22-rdf-syntax-ns#type"]["s"].unique()
nodeTypes

array(['Municipio', 'hotel', 'ComunidadAutonoma', 'Comarca', 'Provincia',
       'empresa-turismo-activo', 'albergue', 'Entity', 'Activity',
       'Agent', 'Organization', 'User_Account', 'Point', 'Voice'],
      dtype=object)

#  NetworkX

<h4> integer data lines supression </h4>

In [11]:
cleandf

,o,p,s
34,Ain,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Municipio
38,hotel-7a2575c0543108e27c8ebf3cdf705d98eb620f18,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,hotel
42,Comunidad_Valenciana,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,ComunidadAutonoma
44,Plana_Baja,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Comarca
46,Castellon,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Provincia
...,...,...,...
815973,AjdeCastellfort,http://krono.act.uji.es/schemas/slodbi#web,www.castellfort.es
815974,AytoBenicassim,http://krono.act.uji.es/schemas/slodbi#web,www.benicassim.org
815975,NulesInfo,http://krono.act.uji.es/schemas/slodbi#web,www.nules.es
815976,AjtVilafames,http://krono.act.uji.es/schemas/slodbi#web,vilafames.org


In [17]:
typeDf=cleandf[cleandf["p"]=="http://www.w3.org/1999/02/22-rdf-syntax-ns#type"]
typeDf

,o,p,s
34,Ain,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Municipio
38,hotel-7a2575c0543108e27c8ebf3cdf705d98eb620f18,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,hotel
42,Comunidad_Valenciana,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,ComunidadAutonoma
44,Plana_Baja,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Comarca
46,Castellon,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Provincia
...,...,...,...
82398,genid-74817f740289417eb03e454f490779cf-E7C843E...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Voice
82399,genid-74817f740289417eb03e454f490779cf-AA792CB...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Point
82400,genid-74817f740289417eb03e454f490779cf-F87FC4E...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Voice
82401,genid-74817f740289417eb03e454f490779cf-7B7D52B...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Point


In [13]:
newdf=cleandf[-(cleandf["p"]=="http://www.w3.org/1999/02/22-rdf-syntax-ns#type")]
newdf

#getting rid of type nodes "Municipio","hotel","Comarca","Provincia"
#"empresa-turismo-activo","albergue" "CunidadAutonoma","Activity","Agent","Organization","User_Account" etc

,o,p,s
82485,Ain,http://www.w3.org/2002/07/owl#sameAs,Q735833
82486,hotel-7a2575c0543108e27c8ebf3cdf705d98eb620f18,http://www.w3.org/2002/07/owl#sameAs,7a2575c0543108e27c8ebf3cdf705d98eb620f18/
82487,Comunidad_Valenciana,http://www.w3.org/2002/07/owl#sameAs,Q5720
82488,Plana_Baja,http://www.w3.org/2002/07/owl#sameAs,Q2300704
82489,Castellon,http://www.w3.org/2002/07/owl#sameAs,Q54942
...,...,...,...
815973,AjdeCastellfort,http://krono.act.uji.es/schemas/slodbi#web,www.castellfort.es
815974,AytoBenicassim,http://krono.act.uji.es/schemas/slodbi#web,www.benicassim.org
815975,NulesInfo,http://krono.act.uji.es/schemas/slodbi#web,www.nules.es
815976,AjtVilafames,http://krono.act.uji.es/schemas/slodbi#web,vilafames.org


# create networkX graph

In [21]:
G=nx.from_pandas_edgelist(newdf, source='s',target='o',edge_attr='p',create_using=nx.DiGraph())
G.size()


# from from 794261 to 711939 with type nodes supression

711939

In [22]:
nx.set_node_attributes(G, typesdict, name="type")

# Graph Properties and Metrics

In [24]:
print("size before : "+str(G.size()))
G.remove_nodes_from(["Municipio","hotel","Comarca","Provincia","empresa-turismo-activo","albergue"])
G.remove_nodes_from(["ComunidadAutonoma","Activity","Agent","Organization","User_Account"])
G.remove_nodes_from(["false","nan","Entiers","true"])
print("size after: "+str(G.size()))

size before : 711922
size after: 595979


In [26]:
predicateNodesDic = dict(filter(lambda elem: elem[1] in ["Entity","Activity","Property"],typesdict.items()))
nodes_to_remove=predicateNodesDic.keys()

print("size before : "+str(G.size()))
G.remove_nodes_from(nodes_to_remove)
print("size after: "+str(G.size()))

size before : 595827
size after: 595827


https://networkx.org/documentation/stable/reference/algorithms/centrality.html NetworkX analysis functions

<h3> Metrics </h3>

In [27]:
density = nx.density(G)
print("Network density:", density)

Network density: 5.787419463729742e-06


In [28]:
triadic_closure = nx.transitivity(G)
print("Triadic closure:", triadic_closure)

Triadic closure: 1.5172693631005608e-06


# Centrality Measures

<h3> generic func </h3>

In [30]:
def node_metric_type_tuple(metric_func):
    to_sort=[]
    for tuple in metric_func(G).items():
        to_sort.append(
            (*tuple,
             (typesdict.get(tuple[0],None))
            )
        )

    sorted_tuples = sorted(to_sort, key=operator.itemgetter(1),reverse=True)
    return sorted_tuples

<h3> degre </h3>

degree_centrality(G)
Compute the degree centrality for nodes.

in_degree_centrality(G)
Compute the in-degree centrality for nodes.

out_degree_centrality(G)
Compute the out-degree centrality for nodes.

Degree centrality is defined as the number of links incident upon a node (i.e., the number of ties that a node has). If the network is directed (meaning that ties have direction), then two separate measures of degree centrality are defined, namely, indegree and outdegree.Degree centrality is defined as the number of links incident upon a node (i.e., the number of ties that a node has). If the network is directed (meaning that ties have direction), then two separate measures of degree centrality are defined, namely, indegree and outdegree.

In [31]:
sorted_degree = sorted(dict(G.degree(G.nodes())).items(), key=operator.itemgetter(1),reverse=True)
sorted_degree
res=[]
for m in sorted_degree:
    res.append((*m,typesdict.get(m[0],None)))
    
res

[('SEGITTUR', 30881, 'User_Account'),
 ('GVAturisme', 7957, 'User_Account'),
 ('VenACastello', 6609, 'User_Account'),
 ('alcossebre_tur', 5866, 'User_Account'),
 ('_peniscola', 5513, 'User_Account'),
 ('tmobenicassim', 3363, 'User_Account'),
 ('España', 2411, None),
 ('Turisme_Vinaros', 2200, 'User_Account'),
 ('morellaturismo', 2185, 'User_Account'),
 ('EcotourismSpain', 1810, 'User_Account'),
 ('0', 1460, None),
 ('castellonenruta', 1375, 'User_Account'),
 ('Madrid', 1365, None),
 ('territorio_es', 1280, 'User_Account'),
 ('1', 1272, None),
 ('TurismeVilareal', 1227, 'User_Account'),
 ('2', 1208, None),
 ('3', 1100, None),
 ('TouristInfoOnda', 1081, 'User_Account'),
 ('Valencia, España', 1080, None),
 ('4', 1035, None),
 ('6', 915, None),
 ('5', 913, None),
 ('ajuntpeniscola', 894, 'User_Account'),
 ('7', 851, None),
 ('Valencia', 846, None),
 ('BurrianaTurismo', 820, 'User_Account'),
 ('8', 807, None),
 ('Barcelona', 795, None),
 ('10', 778, None),
 ('9', 761, None),
 ('TurVilafranc

In [32]:
sorted_degree_centrality=node_metric_type_tuple(nx.degree_centrality)
sorted_degree_centrality

[('SEGITTUR', 0.0962441680353798, 'User_Account'),
 ('GVAturisme', 0.02479890045845397, 'User_Account'),
 ('VenACastello', 0.0205977043018628, 'User_Account'),
 ('alcossebre_tur', 0.018282059832762473, 'User_Account'),
 ('_peniscola', 0.017181894963862857, 'User_Account'),
 ('tmobenicassim', 0.010481174090961507, 'User_Account'),
 ('España', 0.007514157220727979, None),
 ('Turisme_Vinaros', 0.0068565515908757995, 'User_Account'),
 ('morellaturismo', 0.006809802375483465, 'User_Account'),
 ('EcotourismSpain', 0.00564107199067509, 'User_Account'),
 ('0', 0.00455025696485394, None),
 ('castellonenruta', 0.0042853447442973746, 'User_Account'),
 ('Madrid', 0.004254178600702485, None),
 ('territorio_es', 0.00398926638014592, 'User_Account'),
 ('1', 0.003964333465270008, None),
 ('TurismeVilareal', 0.0038240858190930027, 'User_Account'),
 ('2', 0.0037648701462627116, None),
 ('3', 0.0034282757954378997, None),
 ('TouristInfoOnda', 0.0033690601226076086, 'User_Account'),
 ('Valencia, España', 

In [33]:
sorted_in_degree_centrality=node_metric_type_tuple(nx.in_degree_centrality)
sorted_in_degree_centrality

[('Peniscola', 0.000193230090288318, 'Municipio'),
 ('Castello_de_la_Plana', 0.000130897803098538, 'Municipio'),
 ('Benicasim', 0.00012154796002007099, 'Municipio'),
 ('Oropesa_del_Mar', 0.00010596488822262599, 'Municipio'),
 ('Alcala_de_Xivert', 8.1031973346714e-05, 'Municipio'),
 ('Vinaros', 7.7915358987225e-05, 'Municipio'),
 ('Benicarlo', 6.233228718978e-05, 'Municipio'),
 ('Morella', 6.233228718978e-05, 'Municipio'),
 ('hotel-7a2575c0543108e27c8ebf3cdf705d98eb620f18',
  4.6749215392335e-05,
  'hotel'),
 ('hotel-02abccf94c68c48f3ef51d12dd9df2d2caece4c6',
  4.6749215392335e-05,
  'hotel'),
 ('hotel-0adebc3e4867f9d20c19a99d2cf480898d8cc377',
  4.6749215392335e-05,
  'hotel'),
 ('hotel-1fc9e01b356a1262e95a522fd98b3e88df75c3e9',
  4.6749215392335e-05,
  'hotel'),
 ('hotel-5e096f955ed1cbe47030a95b6899bfc8814f2ad6',
  4.6749215392335e-05,
  'hotel'),
 ('hotel-8bb1a63dabb32d79a4f4ba2f62d36fef78030ef2',
  4.6749215392335e-05,
  'hotel'),
 ('hotel-e4923c72d9422e8ca03aeb71755cbdfbcf2ad4ac',


In [34]:
sorted_out_degree_centrality=node_metric_type_tuple(nx.out_degree_centrality)
sorted_out_degree_centrality

#nb arc incident 

[('SEGITTUR', 0.09622546834922287, 'User_Account'),
 ('GVAturisme', 0.024767734314859083, 'User_Account'),
 ('VenACastello', 0.020572771386986887, 'User_Account'),
 ('alcossebre_tur', 0.01825089368916758, 'User_Account'),
 ('_peniscola', 0.017153845434627456, 'User_Account'),
 ('tmobenicassim', 0.010456241176085594, 'User_Account'),
 ('España', 0.007514157220727979, None),
 ('Turisme_Vinaros', 0.006831618675999887, 'User_Account'),
 ('morellaturismo', 0.006784869460607553, 'User_Account'),
 ('EcotourismSpain', 0.005616139075799178, 'User_Account'),
 ('0', 0.00455025696485394, None),
 ('castellonenruta', 0.0042604118294214625, 'User_Account'),
 ('Madrid', 0.004254178600702485, None),
 ('territorio_es', 0.0039674500796294965, 'User_Account'),
 ('1', 0.003964333465270008, None),
 ('TurismeVilareal', 0.003796036289857602, 'User_Account'),
 ('2', 0.0037648701462627116, None),
 ('3', 0.0034282757954378997, None),
 ('Valencia, España', 0.00336594350824812, None),
 ('TouristInfoOnda', 0.003344

<h3> Eigenvector </h3>

In graph theory, eigenvector centrality (also called eigencentrality or prestige score) is a measure of the influence of a node in a network. Relative scores are assigned to all nodes in the network based on the concept that connections to high-scoring nodes contribute more to the score of the node in question than equal connections to low-scoring nodes. A high eigenvector score means that a node is connected to many nodes who themselves have high scores

eigenvector_centrality(G[, max_iter, tol, ...])
Compute the eigenvector centrality for the graph G.

eigenvector_centrality_numpy(G[, weight, ...])
Compute the eigenvector centrality for the graph G.

In graph theory, the Katz centrality of a node is a measure of centrality in a network. It was introduced by Leo Katz in 1953 and is used to measure the relative degree of influence of an actor (or node) within a social network. Unlike typical centrality measures which consider only the shortest path (the geodesic) between a pair of actors, Katz centrality measures influence by taking into account the total number of walks between a pair of actors.

katz_centrality(G[, alpha, beta, max_iter, ...])
Compute the Katz centrality for the nodes of the graph G.

katz_centrality_numpy(G[, alpha, beta, ...])
Compute the Katz centrality for the graph G.

In [35]:
sorted_eigenvector_centrality=node_metric_type_tuple(nx.eigenvector_centrality)
sorted_eigenvector_centrality

#parce que les followers sont connecté a plusieurs noeuds importants ? les comptes de communes etc ? 

[('Benicasim', 0.3916257546054039, 'Municipio'),
 ('jpallaresp', 0.10624414938995805, 'User_Account'),
 ('beni_paraiso', 0.10624414938995805, 'User_Account'),
 ('aquaplana_cs', 0.10624414938995805, 'User_Account'),
 ('MarcosJM4', 0.10624414938995805, 'User_Account'),
 ('RBenicassim', 0.10624414938995805, 'User_Account'),
 ('NeumaticosBenic', 0.10624414938995805, 'User_Account'),
 ('CarmelitanoBD', 0.10624414938995805, 'User_Account'),
 ('CCDBenicassim', 0.10624414938995805, 'User_Account'),
 ('elmuyo', 0.10624414879261952, 'User_Account'),
 ('SoReTaChan', 0.10624414879261952, 'User_Account'),
 ('NazareBenicasim', 0.10624414879261952, 'User_Account'),
 ('hector_bou', 0.10624414879261952, 'User_Account'),
 ('JACF59', 0.10624414879261952, 'User_Account'),
 ('BegoBT', 0.10624414876114302, 'User_Account'),
 ('FerranTarrega', 0.10624414876114302, 'User_Account'),
 ('LaTerracitaBeni', 0.10624414876114302, 'User_Account'),
 ('Martarz79', 0.10624414876114302, 'User_Account'),
 ('AndreeaJ14', 0.

In [36]:
sorted_katz_centrality=node_metric_type_tuple(nx.katz_centrality)
sorted_katz_centrality

[('Peniscola', 0.02062600941537715, 'Municipio'),
 ('Benicasim', 0.015094583536820637, 'Municipio'),
 ('Castello_de_la_Plana', 0.013700672639446245, 'Municipio'),
 ('Oropesa_del_Mar', 0.010955778336457686, 'Municipio'),
 ('Alcala_de_Xivert', 0.007971127362037844, 'Municipio'),
 ('Vinaros', 0.007655214289017509, 'Municipio'),
 ('Morella', 0.006799396756879492, 'Municipio'),
 ('Benicarlo', 0.006104615555234529, 'Municipio'),
 ('Montanejos', 0.005789894122100883, 'Municipio'),
 ('Cabanes', 0.00572469394477121, 'Municipio'),
 ('Onda', 0.005055787055589861, 'Municipio'),
 ('villaespagne', 0.004893096450165751, 'User_Account'),
 ('Pobla_de_Benifassa_la', 0.00485809845351073, 'Municipio'),
 ('Benassal', 0.004630987889011421, 'Municipio'),
 ('Burriana', 0.004622548522445815, 'Municipio'),
 ('AjuntCastello', 0.004393479814893304, 'User_Account'),
 ('jpallaresp', 0.004246816164527159, 'User_Account'),
 ('beni_paraiso', 0.004246816164527159, 'User_Account'),
 ('aquaplana_cs', 0.004246816164527159

<h3> Closeness </h3> 

In a connected graph, closeness centrality (or closeness) of a node is a measure of centrality in a network, calculated as the reciprocal of the sum of the length of the shortest paths between the node and all other nodes in the graph. Thus, the more central a node is, the closer it is to all other nodes.

In [37]:
sorted_closeness_centrality=node_metric_type_tuple(nx.closeness_centrality)
sorted_closeness_centrality

[('Peniscola', 0.001351231612337589, 'Municipio'),
 ('villaespagne', 0.0009771686283653698, 'User_Account'),
 ('Castello_de_la_Plana', 0.0008819293843316779, 'Municipio'),
 ('Benicasim', 0.0007683861385587174, 'Municipio'),
 ('Oropesa_del_Mar', 0.0006492967676105363, 'Municipio'),
 ('AjuntCastello', 0.0006452458246166389, 'User_Account'),
 ('jpallaresp', 0.0005605986017678035, 'User_Account'),
 ('beni_paraiso', 0.0005605986017678035, 'User_Account'),
 ('aquaplana_cs', 0.0005605986017678035, 'User_Account'),
 ('MarcosJM4', 0.0005605986017678035, 'User_Account'),
 ('RBenicassim', 0.0005605986017678035, 'User_Account'),
 ('CarmelitanoBD', 0.0005605986017678035, 'User_Account'),
 ('elmuyo', 0.000560284012204522, 'User_Account'),
 ('SoReTaChan', 0.000560284012204522, 'User_Account'),
 ('NazareBenicasim', 0.000560284012204522, 'User_Account'),
 ('JACF59', 0.000560284012204522, 'User_Account'),
 ('NeumaticosBenic', 0.000559561990386882, 'User_Account'),
 ('CCDBenicassim', 0.000559561990386882

# Graph Modification

In [38]:
df[df["o"].str.match(".*genid-.*")].sample(40)

,o,p,s
254716,genid-74817f740289417eb03e454f490779cf-956E3AD...,http://www.w3.org/2006/vcard/ns#hasValue,tel:617013551
254336,genid-9767df1331394d3ca5d7dae41382a15a-17E4008...,http://www.w3.org/2006/vcard/ns#hasValue,tel:964171180
82201,genid-74817f740289417eb03e454f490779cf-AFD47B7...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2006/vcard/ns#Voice
244943,genid-74817f740289417eb03e454f490779cf-91995B7...,http://www.opengis.net/ont/geosparql#asWKT,POINT(40.369049741000026 0.356765000000052)
250240,genid-74817f740289417eb03e454f490779cf-14E05E0...,http://krono.act.uji.es/schemas/slodbi#locality,genid-74817f740289417eb03e454f490779cf-C25741D...
251374,genid-74817f740289417eb03e454f490779cf-35550A7...,http://www.w3.org/2006/vcard/ns#locality,Oropesa del Mar/Orpesa
249886,genid-9767df1331394d3ca5d7dae41382a15a-BD8EBC5...,http://krono.act.uji.es/schemas/slodbi#locality,genid-9767df1331394d3ca5d7dae41382a15a-01199E1...
250944,genid-9767df1331394d3ca5d7dae41382a15a-1CA44FE...,http://www.w3.org/2006/vcard/ns#locality,Benicàssim/Benicasim
251746,genid-9767df1331394d3ca5d7dae41382a15a-EE43415...,http://www.w3.org/2006/vcard/ns#postal-code,12150
252771,genid-74817f740289417eb03e454f490779cf-84185FC...,http://www.w3.org/2006/vcard/ns#street-address,AV Valencia 7


In [ ]:
df[df["o"].str.match("node")]

for centrality measures etc there are other nodes to get rid off, maybe only include those with a type, to get rid of phone numbers, adresses genid-XXXX or any other non usefull node